# Processing Census 2011 Web Data

In [220]:
import re
import os
import glob
import PyPDF2
import pandas as pd

- move_data_to_one_place
- parse data

In [221]:
Barguna = '../../data/bbs/webtestapplication/userfiles/image/Census2011/Barisal/Barguna/'
baruna_files = sorted(glob.glob(Barguna + '/*[0-9].pdf'))
baruna_files

['../../data/bbs/webtestapplication/userfiles/image/Census2011/Barisal/Barguna/Barguna_C01.pdf',
 '../../data/bbs/webtestapplication/userfiles/image/Census2011/Barisal/Barguna/Barguna_C02.pdf',
 '../../data/bbs/webtestapplication/userfiles/image/Census2011/Barisal/Barguna/Barguna_C03.pdf',
 '../../data/bbs/webtestapplication/userfiles/image/Census2011/Barisal/Barguna/Barguna_C04.pdf',
 '../../data/bbs/webtestapplication/userfiles/image/Census2011/Barisal/Barguna/Barguna_C05.pdf',
 '../../data/bbs/webtestapplication/userfiles/image/Census2011/Barisal/Barguna/Barguna_C06.pdf',
 '../../data/bbs/webtestapplication/userfiles/image/Census2011/Barisal/Barguna/Barguna_C07.pdf',
 '../../data/bbs/webtestapplication/userfiles/image/Census2011/Barisal/Barguna/Barguna_C08.pdf',
 '../../data/bbs/webtestapplication/userfiles/image/Census2011/Barisal/Barguna/Barguna_C09.pdf',
 '../../data/bbs/webtestapplication/userfiles/image/Census2011/Barisal/Barguna/Barguna_C10.pdf',
 '../../data/bbs/webtestapplic

In [359]:
def check_float(string):
    try:
        float(string)
        return True
    except ValueError:
        return False

def parse_files(file, ncol_before, ncol_after, col_names):
    dfFileObj = open(file, 'rb')
    pdfReader = PyPDF2.PdfFileReader(dfFileObj,strict=False)
    num_pages = pdfReader.getNumPages()
    pages = ''
    subset = {}
    for page in range(num_pages):
        pageObj = pdfReader.getPage(page)
        pages = pages + pageObj.extractText()
    pages = pages.split("\n")
    pages = [element.replace(",", "") for element in pages]
    for key, element in enumerate(pages):
        if element.isspace() or not element.strip():
            pages[key] = -99999999
        if key > 6:
            try:
                digits_check = pages[key-4:key+5]
                string_check = digits_check.pop(4)
                digits_check = [int(digit) for digit in digits_check]
                geoname_check = not string_check.isdigit()
                values_check_int = all([str(digit).isdigit() for digit in digits_check])
                # values_check_float = all([check_float(digit) for digit in digits_check])
                # values_check = values_check_float or values_check_int
                geoname_values_check = geoname_check and values_check_int
                patterns = ['Zila', 'Upazila', 'Ward', 'Union', '*']
                pattern_check = any([pattern in element for pattern in patterns])
                if geoname_values_check or pattern_check:
                    subset[key] = pages[key-ncol_before:key+ncol_after]
            except:
                digits_check = pages[key-4:key+5]
                string_check = digits_check.pop(4)
                # digits_check = [check_float(digit) for digit in digits_check]
                geoname_check = not check_float(string_check)
                values_check_float = all([check_float(digit) for digit in digits_check])
                # values_check_int = all([str(digit).isdigit() for digit in digits_check])
                # values_check = values_check_float or values_check_int
                geoname_values_check = geoname_check and values_check_float
                patterns = ['Zila', 'Upazila', 'Ward', 'Union', '*']
                pattern_check = any([pattern in element for pattern in patterns])
                if geoname_values_check or pattern_check:
                    subset[key] = pages[key-ncol_before:key+ncol_after]
            else:
                pass
    subset= pd.DataFrame.from_dict(subset,orient='index', columns=col_names)

    return subset

In [361]:
t1 = parse_files(file=baruna_files[0], 
                     ncol_before=6, 
                     ncol_after=5, 
                     col_names = cols_dict['C01'])
t1.head()

,ZL,UZ,UN/WA,MZ/MH,Vill,RMO,Administrative Unit Residence Community,Area in Acres,Total Households,Population-Total,Population-In Households
16,04,-99999999,-99999999,-99999999,-99999999,-99999999,Barguna Zila Total,452526,215842,892781,891795
29,04,-99999999,-99999999,-99999999,-99999999,1,Barguna Zila,191060,789687,789067,620
40,04,-99999999,-99999999,-99999999,-99999999,2,Barguna Zila,18403,76927,76561,366
51,04,-99999999,-99999999,-99999999,-99999999,3,Barguna Zila,6379,26167,26167,0
62,04,09,-99999999,-99999999,-99999999,-99999999,Amtali Upazila Total,178103,63212,270802,270614


In [362]:
t2 = parse_files(file=baruna_files[1], ncol_before=6, ncol_after=9, col_names = cols_dict['C02'])
t2.head()

,ZL,UZ,UN/WA,MZ/MH,Vill,RMO,Administrative Unit Residence Community,Households-Total,Households-General,Households-Institutional,Households-Others,Population-Both,Population-Male,Population-Female,Sex Ratio
22,04,-99999999,-99999999,-99999999,-99999999,-99999999,Barguna Zila Total,215842,214863,148,831,892781,437413,455368,96
37,04,-99999999,-99999999,-99999999,-99999999,1,Barguna Zila,191060,190564,85,411,789687,385344,404343,95
52,04,-99999999,-99999999,-99999999,-99999999,2,Barguna Zila,18403,17929,60,414,76927,39146,37781,104
67,04,-99999999,-99999999,-99999999,-99999999,3,Barguna Zila,6379,6370,3,6,26167,12923,13244,98
82,04,09,-99999999,-99999999,-99999999,-99999999,Amtali Upazila Total,63212,63066,26,120,270802,132168,138634,95


In [363]:
t3 = parse_files(file=baruna_files[2], ncol_before=6, ncol_after=11, col_names = cols_dict['C03'])
t3.head()

,ZL,UZ,UN/WA,MZ/MH,Vill,RMO,Administrative Unit Residence Community,General Households,Percentage of Households comprising-1 person,Percentage of Households comprising-2 persons,Percentage of Households comprising-3 persons,Percentage of Households comprising-4 persons,Percentage of Households comprising-5 persons,Percentage of Households comprising-6 persons,Percentage of Households comprising-7 persons,Percentage of Households comprising-8+ persons,Average size of Household
25,04,-99999999,-99999999,-99999999,-99999999,-99999999,Barguna Zila Total,214863,3.7,12.5,21.1,26.7,18.5,9.5,4.2,3.8,4.1
42,04,-99999999,-99999999,-99999999,-99999999,1,Barguna Zila,190564,3.6,12.6,20.9,26.5,18.7,9.5,4.3,3.8,4.1
59,04,-99999999,-99999999,-99999999,-99999999,2,Barguna Zila,17929,3.9,11.7,22.9,27.7,16.6,8.8,4.1,4.3,4.1
76,04,-99999999,-99999999,-99999999,-99999999,3,Barguna Zila,6370,3.7,12.3,21.4,27.6,18.1,9.2,4.4,3.5,4.1
93,04,09,-99999999,-99999999,-99999999,-99999999,Amtali Upazila Total,63066,2.7,11.3,19.8,26.4,20.0,10.4,4.8,4.6,4.3


In [364]:
t4 = parse_files(file=baruna_files[3], ncol_before=6, ncol_after=12, col_names = cols_dict['C04'])
t4.head()

,ZL,UZ,UN/WA,MZ/MH,Vill,RMO,Administrative Unit Residence Community,All ages,Percentage of population in the age group 0-4,Percentage of population in the age group 5-9,Percentage of population in the age group 10-14,Percentage of population in the age group 15-19,Percentage of population in the age group 20-24,Percentage of population in the age group 25-29,Percentage of population in the age group 30-49,Percentage of population in the age group 50-59,Percentage of population in the age group 60-64,Percentage of population in the age group 65+
28,04,-99999999,-99999999,-99999999,-99999999,-99999999,Barguna Zila Total,892781,9.9,12.4,11.5,7.1,7.7,9.1,25.6,7.4,3.4,6.0
46,04,-99999999,-99999999,-99999999,-99999999,1,Barguna Zila,789687,10.0,12.6,11.6,6.9,7.7,9.0,25.2,7.4,3.5,6.1
64,04,-99999999,-99999999,-99999999,-99999999,2,Barguna Zila,76927,8.6,10.4,11.1,8.9,8.2,10.3,28.8,6.7,2.6,4.5
82,04,-99999999,-99999999,-99999999,-99999999,3,Barguna Zila,26167,9.4,11.5,11.3,7.5,8.2,9.5,26.0,7.2,3.2,6.2
100,04,09,-99999999,-99999999,-99999999,-99999999,Amtali Upazila Total,270802,10.3,13.2,11.8,6.5,7.7,9.1,25.1,7.1,3.3,5.8


In [365]:
t5 = parse_files(file=baruna_files[4], ncol_before=6, ncol_after=11, col_names = cols_dict['C05'])
t5.head()

,ZL,UZ,UN/WA,MZ/MH,Vill,RMO,Administrative Unit Residence Community,Total Male,% Male-Never married,% Male-Married,% Male-Widowed,% Male-Divorced/Separated,Total Female,% Female-Never married,% Female-Married,% Female-Widowed,% Female-Divorced/Separated
29,04,-99999999,-99999999,-99999999,-99999999,-99999999,Barguna Zila Total,336845,32.8,66.2,0.9,0.1,357200,20.1,70.6,8.7,0.7
46,04,-99999999,-99999999,-99999999,-99999999,1,Barguna Zila,294904,32.4,66.5,1.0,0.1,316200,19.8,70.7,8.8,0.7
63,04,-99999999,-99999999,-99999999,-99999999,2,Barguna Zila,31776,35.9,63.4,0.6,0.1,30481,22.8,69.4,7.2,0.6
80,04,-99999999,-99999999,-99999999,-99999999,3,Barguna Zila,10165,33.0,65.7,1.1,0.2,10519,20.5,69.8,8.9,0.8
97,04,09,-99999999,-99999999,-99999999,-99999999,Amtali Upazila Total,99989,31.9,67.2,0.8,0.1,107242,20.0,71.2,8.2,0.6


In [369]:
print(cols_dict['C06'])
print(len(cols_dict['C06']))
t6 = parse_files(file=baruna_files[5], ncol_before=6, ncol_after=10, col_names = cols_dict['C06'])
t6.head()

['ZL', 'UZ', 'UN/WA', 'MZ/MH', 'Vill', 'RMO', 'Administrative Unit Residence Community', 'Literate (can write a letter)-Both-Yes', 'Literate (can write a letter)-Both-No', 'Literate (can write a letter)-Male-Yes', 'Literate (can write a letter)-Male-No', 'Literate (can write a letter)-Female-Yes', 'Literate (can write a letter)-Female-No', 'Literacy Rate (%)-Both', 'Literacy Rate (%)-Male', 'Literacy Rate (%)-Female']
16


,ZL,UZ,UN/WA,MZ/MH,Vill,RMO,Administrative Unit Residence Community,Literate (can write a letter)-Both-Yes,Literate (can write a letter)-Both-No,Literate (can write a letter)-Male-Yes,Literate (can write a letter)-Male-No,Literate (can write a letter)-Female-Yes,Literate (can write a letter)-Female-No,Literacy Rate (%)-Both,Literacy Rate (%)-Male,Literacy Rate (%)-Female
22,04,-99999999,-99999999,-99999999,-99999999,-99999999,Barguna Zila Total,438254,322121,219357,150965,218897,171156,57.6,59.2,56.1
38,04,-99999999,-99999999,-99999999,-99999999,1,Barguna Zila,375575,295143,186709,138271,188866,156872,56.0,57.5,54.6
54,04,-99999999,-99999999,-99999999,-99999999,2,Barguna Zila,49417,17710,25993,8241,23424,9469,73.6,75.9,71.2
70,04,-99999999,-99999999,-99999999,-99999999,3,Barguna Zila,13262,9268,6655,4453,6607,4815,58.9,59.9,57.8
86,04,09,-99999999,-99999999,-99999999,-99999999,Amtali Upazila Total,120582,107860,60756,49983,59826,57877,52.8,54.9,50.8


In [377]:
print(cols_dict['C07'])
print(len(cols_dict['C07']))
t7 = parse_files(file=baruna_files[6], ncol_before=6, ncol_after=13, col_names = cols_dict['C07'])
t7.head()

['ZL', 'UZ', 'UN/WA', 'MZ/MH', 'Vill', 'RMO', 'Administrative Unit Residence Community', 'Population aged 3-5 years-Attending school-Male', 'Population aged 3-5 years-Attending school-Female', 'Population aged 3-5 years-Not attending school-Male', 'Population aged 3-5 years-Not attending school school-Female', 'Population aged 6-10 years-Attending school-Male', 'Population aged 6-10 years-Attending school-Female', 'Population aged 6-10 years-Not attending school-Male', 'Population aged 6-10 years-Not attending school school-Female', 'Population aged 11-14 years-Attending school-Male', 'Population aged 11-14 years-Attending school-Female', 'Population aged 11-14 years-Not attending school-Male', 'Population aged 11-14 years-Not attending school school-Female']
19


,ZL,UZ,UN/WA,MZ/MH,Vill,RMO,Administrative Unit Residence Community,Population aged 3-5 years-Attending school-Male,Population aged 3-5 years-Attending school-Female,Population aged 3-5 years-Not attending school-Male,Population aged 3-5 years-Not attending school school-Female,Population aged 6-10 years-Attending school-Male,Population aged 6-10 years-Attending school-Female,Population aged 6-10 years-Not attending school-Male,Population aged 6-10 years-Not attending school school-Female,Population aged 11-14 years-Attending school-Male,Population aged 11-14 years-Attending school-Female,Population aged 11-14 years-Not attending school-Male,Population aged 11-14 years-Not attending school school-Female
31,04,-99999999,-99999999,-99999999,-99999999,-99999999,Barguna Zila Total,4547,4744,26972,26472,48628,46865,10019,8947,33750,32069,6824,4192
50,04,-99999999,-99999999,-99999999,-99999999,1,Barguna Zila,3975,4200,24426,23806,43615,42127,9105,8067,29787,28473,6116,3699
69,04,-99999999,-99999999,-99999999,-99999999,2,Barguna Zila,410,391,1845,1937,3647,3435,683,673,2991,2656,503,385
88,04,-99999999,-99999999,-99999999,-99999999,3,Barguna Zila,162,153,701,729,1366,1303,231,207,972,940,205,108
107,04,09,-99999999,-99999999,-99999999,-99999999,Amtali Upazila Total,1253,1332,8766,8566,15325,14745,3660,3355,9906,9524,2330,1586


In [380]:
print(cols_dict['C08'])
print(len(cols_dict['C08']))
t8 = parse_files(file=baruna_files[7], ncol_before=6, ncol_after=13, col_names = cols_dict['C08'])
t8.head()

['ZL', 'UZ', 'UN/WA', 'MZ/MH', 'Vill', 'RMO', 'Administrative Unit Residence Community', 'Population aged 15-19 years-Attending school-Male', 'Population aged 15-19 years-Attending school-Female', 'Population aged 15-19 years-Not attending school-Male', 'Population aged 15-19 years-Not attending school school-Female', 'Population aged 20-24 years-Attending school-Male', 'Population aged 20-24 years-Attending school-Female', 'Population aged 20-24 years-Not attending school-Male', 'Population aged 20-24 years-Not attending school school-Female', 'Population aged 25-29 years-Attending school-Male', 'Population aged 25-29 years-Attending school-Female', 'Population aged 25-29 years-Not attending school-Male', 'Population aged 25-29 years-Not attending school school-Female']
19


,ZL,UZ,UN/WA,MZ/MH,Vill,RMO,Administrative Unit Residence Community,Population aged 15-19 years-Attending school-Male,Population aged 15-19 years-Attending school-Female,Population aged 15-19 years-Not attending school-Male,Population aged 15-19 years-Not attending school school-Female,Population aged 20-24 years-Attending school-Male,Population aged 20-24 years-Attending school-Female,Population aged 20-24 years-Not attending school-Male,Population aged 20-24 years-Not attending school school-Female,Population aged 25-29 years-Attending school-Male,Population aged 25-29 years-Attending school-Female,Population aged 25-29 years-Not attending school-Male,Population aged 25-29 years-Not attending school school-Female
31,04,-99999999,-99999999,-99999999,-99999999,-99999999,Barguna Zila Total,17216,13999,15477,16933,3420,2043,22943,40651,695,291,32746,47401
50,04,-99999999,-99999999,-99999999,-99999999,1,Barguna Zila,14388,11463,13769,15185,2703,1474,20274,36172,513,188,28417,41645
69,04,-99999999,-99999999,-99999999,-99999999,2,Barguna Zila,2273,2036,1252,1291,613,475,1958,3255,156,87,3312,4337
88,04,-99999999,-99999999,-99999999,-99999999,3,Barguna Zila,555,500,456,457,104,94,711,1224,26,16,1017,1419
107,04,09,-99999999,-99999999,-99999999,-99999999,Amtali Upazila Total,4445,3376,4627,5187,845,485,6925,12572,181,88,9908,14583


In [384]:
print(cols_dict['C09'])
print(len(cols_dict['C09']))
t9 = parse_files(file=baruna_files[8], ncol_before=6, ncol_after=9, col_names = cols_dict['C09'])
t9.head()

['ZL', 'UZ', 'UN/WA', 'MZ/MH', 'Vill', 'RMO', 'Administrative Unit Residence Community', 'Total Population', 'Type of disability (%)-All', 'Type of disability (%)-Speech', 'Type of disability (%)-Vision', 'Type of disability (%)-Hearing', 'Type of disability (%)-Physical', 'Type of disability (%)-Mental', 'Type of disability (%)-Autism']
15


,ZL,UZ,UN/WA,MZ/MH,Vill,RMO,Administrative Unit Residence Community,Total Population,Type of disability (%)-All,Type of disability (%)-Speech,Type of disability (%)-Vision,Type of disability (%)-Hearing,Type of disability (%)-Physical,Type of disability (%)-Mental,Type of disability (%)-Autism
22,04,-99999999,-99999999,-99999999,-99999999,-99999999,Barguna Zila Total,892781,2.1,0.3,0.4,0.2,0.9,0.2,0.1
37,04,-99999999,-99999999,-99999999,-99999999,1,Barguna Zila,789687,2.2,0.3,0.4,0.2,0.9,0.2,0.1
52,04,-99999999,-99999999,-99999999,-99999999,2,Barguna Zila,76927,1.3,0.2,0.3,0.1,0.5,0.2,0.1
67,04,-99999999,-99999999,-99999999,-99999999,3,Barguna Zila,26167,2.5,0.2,0.5,0.2,1.2,0.2,0.2
82,04,09,-99999999,-99999999,-99999999,-99999999,Amtali Upazila Total,270802,2.0,0.3,0.4,0.2,0.9,0.2,0.1


In [382]:
cols_dict = {'C01': ['ZL', 'UZ', 'UN/WA', 'MZ/MH', 'Vill', 'RMO',
                     'Administrative Unit Residence Community','Area in Acres',
                     'Total Households', 'Population-Total','Population-In Households'],
             'C02': ['ZL', 'UZ', 'UN/WA', 'MZ/MH', 'Vill', 'RMO',
                     'Administrative Unit Residence Community',
                     'Households-Total', 'Households-General','Households-Institutional',
                      'Households-Others', 'Population-Both', 'Population-Male', 'Population-Female', 'Sex Ratio'],
             'C03': ['ZL', 'UZ', 'UN/WA', 'MZ/MH', 'Vill', 'RMO',
                     'Administrative Unit Residence Community',
                     'General Households', 'Percentage of Households comprising-1 person',
                     'Percentage of Households comprising-2 persons','Percentage of Households comprising-3 persons',
                     'Percentage of Households comprising-4 persons','Percentage of Households comprising-5 persons',
                     'Percentage of Households comprising-6 persons', 'Percentage of Households comprising-7 persons',
                     'Percentage of Households comprising-8+ persons', 'Average size of Household'],
             'C04': ['ZL', 'UZ', 'UN/WA', 'MZ/MH', 'Vill', 'RMO',
                     'Administrative Unit Residence Community', 'All ages',
                     'Percentage of population in the age group 0-4',
                     'Percentage of population in the age group 5-9','Percentage of population in the age group 10-14',
                     'Percentage of population in the age group 15-19','Percentage of population in the age group 20-24',
                     'Percentage of population in the age group 25-29', 'Percentage of population in the age group 30-49',
                     'Percentage of population in the age group 50-59', 'Percentage of population in the age group 60-64',
                    'Percentage of population in the age group 65+',],
             'C05': ['ZL', 'UZ', 'UN/WA', 'MZ/MH', 'Vill', 'RMO',
                     'Administrative Unit Residence Community',
                     'Total Male', '% Male-Never married', '% Male-Married', '% Male-Widowed','% Male-Divorced/Separated',
                     'Total Female', '% Female-Never married','% Female-Married', '% Female-Widowed', '% Female-Divorced/Separated'],
             'C06': ['ZL', 'UZ', 'UN/WA', 'MZ/MH', 'Vill', 'RMO',
                     'Administrative Unit Residence Community',
                     'Literate (can write a letter)-Both-Yes', 'Literate (can write a letter)-Both-No',
                     'Literate (can write a letter)-Male-Yes', 'Literate (can write a letter)-Male-No',
                     'Literate (can write a letter)-Female-Yes', 'Literate (can write a letter)-Female-No',
                     'Literacy Rate (%)-Both', 'Literacy Rate (%)-Male', 'Literacy Rate (%)-Female',],
             'C07': ['ZL', 'UZ', 'UN/WA', 'MZ/MH', 'Vill', 'RMO',
                     'Administrative Unit Residence Community',
                     'Population aged 3-5 years-Attending school-Male','Population aged 3-5 years-Attending school-Female',
                     'Population aged 3-5 years-Not attending school-Male','Population aged 3-5 years-Not attending school school-Female',
                     'Population aged 6-10 years-Attending school-Male','Population aged 6-10 years-Attending school-Female',
                     'Population aged 6-10 years-Not attending school-Male','Population aged 6-10 years-Not attending school school-Female',
                     'Population aged 11-14 years-Attending school-Male','Population aged 11-14 years-Attending school-Female',
                     'Population aged 11-14 years-Not attending school-Male','Population aged 11-14 years-Not attending school school-Female',],
             'C08': ['ZL', 'UZ', 'UN/WA', 'MZ/MH', 'Vill', 'RMO',
                     'Administrative Unit Residence Community',
                     'Population aged 15-19 years-Attending school-Male','Population aged 15-19 years-Attending school-Female',
                     'Population aged 15-19 years-Not attending school-Male','Population aged 15-19 years-Not attending school school-Female',
                     'Population aged 20-24 years-Attending school-Male','Population aged 20-24 years-Attending school-Female',
                     'Population aged 20-24 years-Not attending school-Male','Population aged 20-24 years-Not attending school school-Female',
                     'Population aged 25-29 years-Attending school-Male','Population aged 25-29 years-Attending school-Female',
                     'Population aged 25-29 years-Not attending school-Male','Population aged 25-29 years-Not attending school school-Female',],

             'C09': ['ZL', 'UZ', 'UN/WA', 'MZ/MH', 'Vill', 'RMO',
                     'Administrative Unit Residence Community', 
                     'Total Population', 'Type of disability (%)-All', 'Type of disability (%)-Speech',
                     'Type of disability (%)-Vision', 'Type of disability (%)-Hearing',
                     'Type of disability (%)-Physical', 'Type of disability (%)-Mental', 
                     'Type of disability (%)-Autism',],
             'C10': ['ZL', 'UZ', 'UN/WA', 'MZ/MH', 'Vill', 'RMO',
                     'Administrative Unit Residence Community',],
             'C11': ['ZL', 'UZ', 'UN/WA', 'MZ/MH', 'Vill', 'RMO',
                     'Administrative Unit Residence Community',],
             'C12': ['ZL', 'UZ', 'UN/WA', 'MZ/MH', 'Vill', 'RMO',
                     'Administrative Unit Residence Community',],
            }